In [1]:
from fomo.pipelines.train import Learner
from fomo.pipelines.types.learner_args import LearnerArgs
from fomo.utils.data.datasets import DatasetInitializer, PROMPTS
import torch

/home/jupyter/github/fomo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NUM_OF_SAMPLES = 16

In [3]:
#TRAIN_SUBSAMPLES = ["base", "all"]
#TRAIN_SUBSAMPLES = ["base"]
TRAIN_SUBSAMPLES = ["base"]
MODEL_TYPE = "clip_transformer_adapter_text"


In [4]:
#DATASETS = ["cifar10", "caltech101", "oxford_pets", "oxford_flowers", "food101", "stanford_cars", "fgvc_aircraft", "dtd", "sun397"]
#DATASETS = ["caltech101", "oxford_pets", "oxford_flowers", "food101", "stanford_cars", "fgvc_aircraft", "dtd", "sun397"]
#DATASETS = ["caltech101", "oxford_pets", "oxford_flowers", "fgvc_aircraft"]
DATASETS = ["oxford_flowers"]
#DATASETS = ["food101", "stanford_cars", "dtd", "sun397", "eurosat", "ucf101"]

In [6]:
with open("classnames.txt") as f:
    CLASS_NAMES = [" ".join(val.split()[1:]) for val in f.read().splitlines()]

CLASS_NAMES

['tench',
 'goldfish',
 'great white shark',
 'tiger shark',
 'hammerhead shark',
 'electric ray',
 'stingray',
 'rooster',
 'hen',
 'ostrich',
 'brambling',
 'goldfinch',
 'house finch',
 'junco',
 'indigo bunting',
 'American robin',
 'bulbul',
 'jay',
 'magpie',
 'chickadee',
 'American dipper',
 'kite (bird of prey)',
 'bald eagle',
 'vulture',
 'great grey owl',
 'fire salamander',
 'smooth newt',
 'newt',
 'spotted salamander',
 'axolotl',
 'American bullfrog',
 'tree frog',
 'tailed frog',
 'loggerhead sea turtle',
 'leatherback sea turtle',
 'mud turtle',
 'terrapin',
 'box turtle',
 'banded gecko',
 'green iguana',
 'Carolina anole',
 'desert grassland whiptail lizard',
 'agama',
 'frilled-necked lizard',
 'alligator lizard',
 'Gila monster',
 'European green lizard',
 'chameleon',
 'Komodo dragon',
 'Nile crocodile',
 'American alligator',
 'triceratops',
 'worm snake',
 'ring-necked snake',
 'eastern hog-nosed snake',
 'smooth green snake',
 'kingsnake',
 'garter snake',
 'w

In [5]:
for train_subsample in TRAIN_SUBSAMPLES:
    for dataset_name in DATASETS:
        print(f"Training {MODEL_TYPE} on {dataset_name} on {train_subsample} subsample.")

        _dataset = DatasetInitializer.from_str(dataset_name).value(train=True)
        labels = _dataset.labels
        prompt_template = PROMPTS[dataset_name]
        num_of_classes = len(labels) // 2 if train_subsample == "base" else len(labels)
        #num_of_classes = len(labels) - (len(labels) // 5) if train_subsample == "base" else len(labels)
        train_eval_size = [NUM_OF_SAMPLES * num_of_classes, NUM_OF_SAMPLES * num_of_classes]

        learner_args = LearnerArgs(model_type=MODEL_TYPE, dataset=dataset_name)
        learner_args.device = "cuda"
        learner_args.epochs = 200
        learner_args.model_type = MODEL_TYPE
        learner_args.train_eval_size = train_eval_size
        learner_args.text_prompt_template = prompt_template
        learner_args.print_freq = 200
        learner_args.save_freq = 200
        learner_args.learning_rate = 5e-3
        learner_args.use_wandb = True
        learner_args.patience = 20
        learner_args.info = "test"
        learner_args.batch_size = 256
        learner_args.momentum = 0.5
        learner_args.train_subsample = train_subsample
        learner_args.dataset = dataset_name

        learner = Learner(learner_args)
        learner.run()

Training clip_transformer_adapter_text on oxford_flowers on base subsample.


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Turning off gradients in both the image and the text encoder
Parameters to be updated: {'adapter.mha.out_proj.bias', 'adapter.downsampler.1.bias', 'adapter.mha.out_proj.weight', 'adapter.downsampler.0.weight', 'adapter.downsampler.0.bias', 'adapter.downsampler.1.weight', 'adapter.upsampler.0.bias', 'adapter.mha.in_proj_weight', 'adapter.upsampler.2.weight', 'adapter.upsampler.2.bias', 'adapter.mha.in_proj_bias', 'adapter.upsampler.0.weight'}
Number of learnable paramms: 107712


wandb: Currently logged in as: dqmiss. Use `wandb login --relogin` to force relogin


  0%|          | 0/4 [00:00<?, ?it/s]

...

 * Prompt Acc@1 75.973


 17%|█▋        | 1/6 [00:04<00:21,  4.40s/it]

Validate: [0/6]	Time  4.396 ( 4.396)	Loss 1.5499e+00 (1.5499e+00)	Prompt Acc@1  73.44 ( 73.44)


100%|██████████| 6/6 [00:08<00:00,  1.37s/it]

 * Prompt Acc@1 75.319
